In [47]:
from ast import literal_eval
from pprint import pprint as pp
import collections
import multiprocessing
import sys
import warnings

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import dask.dataframe as dd

%matplotlib inline
pd.set_option('max_columns', 999)
warnings.filterwarnings('ignore')

N_CPUS = multiprocessing.cpu_count()

In [2]:
train_df = pd.read_csv('data/train.csv')
print(train_df.shape)

train_df.head()

(11341042, 11)


,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,45bb1e1b6b50c07b,2019-09-06T17:53:46.937Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,17eeb7f223665f53,2019-09-06T17:54:17.519Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,77261ab5,0848ef14a8dc6892,2019-09-06T17:54:56.302Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0001e90f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK
3,b2dba42b,0848ef14a8dc6892,2019-09-06T17:54:56.387Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,2,3010,53,Sandcastle Builder (Activity),Activity,MAGMAPEAK
4,1bb5fbdb,0848ef14a8dc6892,2019-09-06T17:55:03.253Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,3,3110,6972,Sandcastle Builder (Activity),Activity,MAGMAPEAK


In [11]:
train_df.head(5).event_data.apply(literal_eval).tolist()

[{'event_code': 2000, 'event_count': 1},
 {'event_code': 2000, 'event_count': 1},
 {'version': '1.0', 'event_count': 1, 'game_time': 0, 'event_code': 2000},
 {'description': "Let's build a sandcastle! First, fill up your mold with sand! You can use the shovel here. The mold gives the sand its shape!",
  'identifier': 'Dot_LetsSandcastle,Dot_FillMold,Dot_MoldShape',
  'media_type': 'audio',
  'total_duration': 6758,
  'event_count': 2,
  'game_time': 53,
  'event_code': 3010},
 {'description': "Let's build a sandcastle! First, fill up your mold with sand! You can use the shovel here. The mold gives the sand its shape!",
  'identifier': 'Dot_LetsSandcastle,Dot_FillMold,Dot_MoldShape',
  'media_type': 'audio',
  'duration': 6919,
  'event_count': 3,
  'game_time': 6972,
  'event_code': 3110}]

In [ ]:
def flatten_dict(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

def process_event_data(event_data):
    # relies on literal_eval, need str
    assert isinstance(event_data, str) 
    try:
        event_data = flatten_dict(literal_eval(event_data))
        event_data['bad_string'] = False
    except ValueError:
        event_data = dict(bad_string=True)
    return event_data

def organize_events_data(df):
    df = dd.from_pandas(df, npartitions=N_CPUS)
    events_list = df.event_data.apply(process_event_data).compute().tolist()
    events_df = pd.DataFrame({k: v for k,v in enumerate(events_list)}).T
    return events_df


%time train_events_df = organize_events_data(train_df)
print(train_events_df.shape)
print(train_events_df.loc[train_events_df.bad_string==True].shape)
train_events_df.head()

# _ = pd.DataFrame({k: v for k,v in enumerate(train_df.head(250).event_data.apply(process_event_data).tolist())}).T
# _.loc[_.bad_string==True]

/Users/washburp/anaconda3/lib/python3.6/site-packages/dask/dataframe/core.py:2285: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  warnings.warn(msg)


In [ ]:
del _
sys.stdout.flush()

In [ ]:
# def flatten_dict(d, parent_key='', sep='_'):
#     items = []
#     for k, v in d.items():
#         new_key = parent_key + sep + k if parent_key else k
#         if isinstance(v, collections.MutableMapping):
#             items.extend(flatten_dict(v, new_key, sep=sep).items())
#         else:
#             items.append((new_key, v))
#     return dict(items)

# def process_event_data(event_data):
#     try:
#         event_data = flatten_dict(literal_eval(str(event_data)))
#     except ValueError:
#         print('Issue with string compatability with literal_eval')
#         print(event_data)
#         break
#         event_data = dict()
#     return event_data

# def organize_events_data(df):
#     events_df = pd.DataFrame({k: v for k,v in enumerate(df.event_data.apply(process_event_data).tolist())}).T

# train_events_df = organize_events_data(train_df)
# print(train_events_df.shape)
# train_events_df.head()

In [ ]:
def preprocess(df):
    